# Get Babel Users

In [23]:
from lxml import etree
import re
import pandas as pd

babelreg = re.compile('{{Babel(.*)}}')

def process_element(elem, lang):
    langreg = '%s-(.)' % lang
    if elem[1].text == '2':
        text = elem[-1][-2].text
        match = None
        if text is not None:
            match = babelreg.search (text)
        if match is not None:
            for g in match.groups():
                match = re.search(langreg, g)
                if match is not None:
                    user =  elem[0].text.split(':')[1]
                    proficiency = match.groups()[0]
                    return (user, proficiency )
    return None


def find_babel(dump, lang):
    context = etree.iterparse( dump , tag = '{http://www.mediawiki.org/xml/export-0.10/}page')
    l = []
    for event, elem in context:
        user = process_element(elem, lang)
        if user is not None:
            l.append(user)
        elem.clear()
        for ancestor in elem.xpath('ancestor-or-self::*'):
            while ancestor.getprevious() is not None:
                del ancestor.getparent()[0]
    del context
    return l

def worker(t):
    return find_babel(t[0], t[1])

def find_babel_parallel(wikilang, babellang, n):
    exp_dir = 'en-%s' % wikilang
    args = [('/home/ellery/%s/dump%d.xml' % (exp_dir, i), babellang) for i in range(1, n+1)]
    p = multiprocessing.Pool(8)
    results = p.map(worker, args)
    l = list(itertools.chain(*results))
    df = pd.DataFrame(l)
    df.columns = ['user', 'proficiency']
    return df
    

In [24]:
import multiprocessing
import itertools

s = 'en'
t = 'es'
min_date = '20140600000000'
min_bytes = 100.0
max_edit_history = 100

In [25]:
t_users = find_babel_parallel(t,s,4)
t_users = set(t_users[t_users['proficiency'] > '1']['user'])

#WE DON"T WANT EN USERS AFTERALL
#s_users = find_babel_parallel(s,t, 26)
#s_users = set(s_users[s_users['proficiency'] > '1']['user'])
#s_users = s_users.difference(t_users)

In [26]:
print len(t_users)
#print len(s_users)

1045


# Get Overlapping Usernames

In [27]:
from ConfigParser import SafeConfigParser
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
from util import save_rdd, get_parser
import json

config = '/home/ellery/wikimedia/missing_articles/missing_articles.ini'
cp = SafeConfigParser()
cp.read(config)

['/home/ellery/wikimedia/missing_articles/missing_articles.ini']

In [28]:
#find editors who have edited both source and target
source_contributions_file = os.path.join(cp.get('general', 'contributions_dir'), s)
target_contributions_file = os.path.join(cp.get('general', 'contributions_dir'), t)

names = ["language_code", "user_id", "user", "id","page_title","num_edits","timestamp", "bytes_added"]

recent_source_users = sc.textFile(source_contributions_file)\
.map(get_parser(names))\
.filter(lambda x: len(x) == len(names))\
.filter(lambda x: x['timestamp'] > min_date)\
.map(lambda x: x['user'])\
.distinct().collect()

source_users = sc.textFile(source_contributions_file)\
.map(get_parser(names))\
.filter(lambda x: len(x) == len(names))\
.map(lambda x: x['user'])\
.distinct().collect()

target_users = sc.textFile(target_contributions_file)\
.map(get_parser(names))\
.filter(lambda x: len(x) == len(names))\
.map(lambda x: x['user'])\
.distinct().collect()

recent_target_users = sc.textFile(target_contributions_file)\
.map(get_parser(names))\
.filter(lambda x: len(x) == len(names))\
.filter(lambda x: x['timestamp'] > min_date)\
.map(lambda x: x['user'])\
.distinct().collect()

In [29]:
o_users = set(recent_source_users).intersection(set(target_users))
o_users = o_users.union(set(recent_target_users).intersection(set(source_users)))
o_users = o_users.difference(t_users)
#o_users = o_users.difference(t_users.union(s_users))

In [30]:
print len(o_users)

28463


# Get User Emails

In [31]:
import pymysql

def query_db(query, params):
    conn = pymysql.connect(host = 'analytics-store.eqiad.wmnet', read_default_file="/etc/mysql/conf.d/analytics-research-client.cnf")
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    cursor.execute(query, params)
    rows = cursor.fetchall()
    conn.close()
    return mysql_to_pandas(rows)


def mysql_to_pandas(dicts):
    dmaster = {}
    for d in dicts:
        for k in d.keys():
            if k not in dmaster:
                dmaster[k] = []
            
            dmaster[k].append(d[k]) 
    return pd.DataFrame(dmaster)

def decode(x):
    try:
         return x.decode('utf-8')
    except:
        return x


In [32]:
query = """
SELECT user_name, user_email
FROM %(lang)swiki.user 
WHERE  length(user_email) > 1 
AND user_editcount > 0 
AND user_email_authenticated is not NULL
"""

In [33]:
t_email_df = query_db(query % {'lang':t}, {})
t_email_df.rename(columns={'user_name':'user'}, inplace=True)
t_email_df['user'] = t_email_df['user'].apply(decode)

In [34]:
s_email_df = query_db(query % {'lang':s}, {})
s_email_df.rename(columns={'user_name':'user'}, inplace=True)
s_email_df['user'] = s_email_df['user'].apply(decode)

# Get Edit History and Email

In [35]:
def to_str(t):
    uid, contributions = t
    contributions = list(contributions)
    uname = contributions[0]['user']
    contributions = [{k: d[k] for k in ('language_code', 'id', 'page_title', 'num_edits', 'timestamp', 'bytes_added')} for d in contributions]
    contributions.sort(key = lambda x: x['timestamp'])
    if contributions[-1]['timestamp'] < min_date:
        return None
    else: 
        obj = {'uid': uid, 'uname': uname, 'contributions': contributions[-max_edit_history:]}
        return (uname, json.dumps(obj))

def get_contributions(cp, langs, user_set):
    contributions_file = os.path.join(cp.get('general', 'contributions_dir'), langs[0])
    contributions = sc.textFile(contributions_file)
    for lang in langs[1:]:
        contributions_file = os.path.join(cp.get('general', 'contributions_dir'), lang)
        contributions = contributions.union(sc.textFile(contributions_file))
        
    contributions = contributions.map(get_parser(names))\
    .filter(lambda x: len(x) == 8)\
    .filter(lambda x: x['user'] in user_set)\
    .filter(lambda x: float(x['bytes_added']) > min_bytes)\
    .map(lambda x: (x['user'], x)).groupByKey()\
    .map(to_str)\
    .filter(lambda x: x is not None)\
    .collect()
    contributions = pd.DataFrame(contributions)
    contributions.columns =['user', 'history']
    return contributions

In [36]:
# eswiki babel
t_users_df = get_contributions(cp, [t], t_users)
t_users_df['user'] = t_users_df['user'].apply(decode)
t_users_df = t_users_df.merge(t_email_df, how='inner', on='user', suffixes=('_x', '_y'), copy=True)

In [37]:
# enwiki babel
#s_users_df = get_contributions(cp, [s], s_users)
#print s_users_df.shape
#s_users_df['user'] = s_users_df['user'].apply(decode)
#s_users_df = s_users_df.merge(s_email_df, how='inner', on='user', suffixes=('_x', '_y'), copy=True)
#print s_users_df.shape

In [38]:
# overlap
o_users_df = get_contributions(cp, [s, t], o_users)
o_users_df['user'] = o_users_df['user'].apply(decode)
o_users_df = o_users_df.merge(s_email_df, how='left', on='user', copy=True)
o_users_df = o_users_df.merge(t_email_df, how='left', on='user', suffixes=('_x', '_t'), copy=True)
o_users_df = o_users_df[o_users_df['user_email_x'] == o_users_df['user_email_t']]
del o_users_df['user_email_t']
o_users_df.rename(columns={'user_email_x':'user_email'}, inplace=True)

In [39]:
t_split = int(t_users_df.shape[0]/2.0)
#s_split = int(s_users_df.shape[0]/2.0)
o_split = int(o_users_df.shape[0]/2.0)


In [40]:
t_users_df1 = t_users_df[:t_split]
t_users_df2 = t_users_df[t_split:]

#s_users_df1 = s_users_df[:s_split]
#s_users_df2 = s_users_df[s_split:]

o_users_df1 = o_users_df[:o_split]
o_users_df2 = o_users_df[o_split:]

In [41]:
df1 = pd.concat([t_users_df1, o_users_df1] )
df2 = pd.concat([t_users_df2, o_users_df2] )

In [42]:
df1.to_csv('/home/ellery/en-%s/e1.tsv' % t,  sep = '\t', encoding = 'utf8')
df2.to_csv('/home/ellery/en-%s/e2.tsv' % t,  sep = '\t', encoding = 'utf8')

In [43]:
df1.shape

(4325, 3)

In [44]:
df2.shape

(4325, 3)